In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
openai.api_key = os.getenv("OPENAI_API_KEY")

# Select the model you want to use

In [ ]:
from langchain.chat_models import ChatOpenAI

model = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.0)


# Use Sequential Chains to build a customer service agent

In [ ]:
# fill in your review here
review = """
I was on the fence for a long time about getting these. The price was definitely a scare factor. However I recently bought an Apple Watch and the headphones compliment the watch perfectly. Especially for exercise. The AirPods Pro sound very good for their size. Battery life is acceptable and I can easily connect them to my Apple TV, Watch, iPhone etc…
They stay in place (I’m a commuter cyclist) with no issues and have never fallen out at the gym either. I will say the volume control on mine is quite finicky but I don’t know if that’s an isolated issue with mine. I love how loud these get as well as the ability to bring the charging case to ‘top up’ the AirPod Pros on the go. If using with an iPhone, you can wander a fair distance away before any connection issues happen. Which is great. Cleaning them is a breeze too. By far my favourite feature is the noise cancelling option. Pretty much worth the price of admission by its self."""

In [ ]:
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain, SequentialChain
from langchain.output_parsers import (
    ResponseSchema,
    StructuredOutputParser,
)


template="""Interprete the text and evaluate the text.
sentiment: is the text positive, negative or neutral?
subject: What is the subject of the text? Use exactly one word not two or more.

Format the output as JSON with the following keys:
sentiment
subject
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

sentiment_schema = ResponseSchema(name="sentiment", description="Is the text positive, negative or neutral? only provide these word")
subject_schema = ResponseSchema(name="subject", description="What is the subject of the text? Use exactly one word.")
# price_schema = ResponseSchema(name="price", description="What is the price of the product?")

response_schemas = [sentiment_schema, subject_schema]

parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()



# using a chain to run the prompt
chain = LLMChain(llm=model, prompt=chat_prompt)
response = chain.run(text=review)



# response_template = """
# You are a helpful bot that creates a 'thank you' reponse text. 
# If customers are unsatisfied, offer them a real world assitant to talk to. 
# You will get a sentiment and subject as into and evaluate. 

# text: {input}
# """

# response_system_message_prompt = SystemMessagePromptTemplate.from_template(response_template)
# response_human_message_prompt = HumanMessagePromptTemplate.from_template(response)



In [ ]:
output_dict = parser.parse(response)
output_dict